## A.1 Libraries

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
idx = pd.IndexSlice

# 1. Consumer spending data

## 1.A FIPS Codes

In [2]:
url = 'https://www2.census.gov/programs-surveys/popest/geographies/2018/state-geocodes-v2018.xlsx'
state_code = pd.read_excel(url, header=5)
state_code = state_code.loc[state_code.iloc[:,-2]!=0]
map_data = state_code.set_index("State (FIPS)").iloc[:,-1].to_dict()

/Users/corybaird/opt/miniconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


## 1.B Read OI Data

In [3]:
def read_oidata(url):
    df = pd.read_csv(url)
    df['date'] = pd.to_datetime(df[['year','month','day']])
    df.set_index('date',inplace=True)
    df = df.replace('.',np.NaN).dropna()
    df['state'] = df.statefips.map(map_data) # Map data from section A
    return df

## 1.1 Import data

In [4]:
url = 'https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Affinity%20-%20State%20-%20Daily.csv'
df = read_oidata(url)
df.head(1)

,year,month,day,statefips,freq,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,...,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4,provisional,state
date,,,,,,,,,,,,,,,,,,,,,
2020-01-13,2020,1,13,1,d,.0133,.0518,-.0216,.0909,.0241,...,.00654,.00376,.00131,.0109,.0113,.025,-.00945,.0201,0,Alabama


## 1.2 Manipulate data

In [5]:
df = df.drop(['year','month','day','statefips','freq', 'provisional'], axis=1)
df = df.reset_index().set_index(['state','date'])
df = df.astype(float)
df = df.sort_index(0)
df.head()

<ipython-input-5-c12d68581165>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  df = df.sort_index(0)


spend_all  spend_aap  spend_acf  spend_aer  spend_apg  \
state   date                                                                
Alabama 2020-01-13    0.01330     0.0518   -0.02160     0.0909    0.02410   
        2020-01-14   -0.00345     0.0993   -0.00586     0.0810    0.03400   
        2020-01-15    0.00466     0.0841    0.00702     0.0937    0.02600   
        2020-01-16    0.01140     0.0857    0.00710     0.0616    0.03800   
        2020-01-17   -0.00428     0.0237    0.01520     0.0620   -0.00127   

                    spend_durables  spend_nondurables  spend_grf  spend_gen  \
state   date                                                                  
Alabama 2020-01-13        -0.00813           0.012600   0.010400    -0.0106   
        2020-01-14        -0.03270           0.000144  -0.008730    -0.0407   
        2020-01-15        -0.03120          -0.003490  -0.010700    -0.0385   
        2020-01-16        -0.00866           0.007130  -0.000508    -0.0190   
        2020-01-17        -0.01200          -0.028700  -0.042900    -0.0237   

                    spend_hic  ...  spend_remoteservices  spend_sgh  \
state   date                   ...                                    
Alabama 2020-01-13   -0.02130  ...                0.0683    0.00473   
        2020-01-14   -0.06970  ...                0.1130   -0.01930   
        2020-01-15   -0.04070  ...                0.1400    0.01040   
        2020-01-16    0.02260  ...                0.1070   -0.04330   
        2020-01-17   -0.00356  ...                0.0771   -0.05450   

                    spend_tws  spend_retail_w_grocery  \
state   date                                            
Alabama 2020-01-13    0.00654                 0.00376   
        2020-01-14   -0.02550                -0.01250   
        2020-01-15    0.00877                -0.01430   
        2020-01-16    0.01690                 0.00416   
        2020-01-17   -0.00196                -0.01710   

                    spend_retail_no_grocery  spend_all_incmiddle  \
state   date                                                       
Alabama 2020-01-13                  0.00131              0.01090   
        2020-01-14                 -0.01450             -0.00634   
        2020-01-15                 -0.01700              0.00316   
        2020-01-16                  0.00530              0.00696   
        2020-01-17                 -0.00498             -0.00789   

                    spend_all_q1  spend_all_q2  spend_all_q3  spend_all_q4  
state   date                                                                
Alabama 2020-01-13       0.01130      0.025000      -0.00945       0.02010  
        2020-01-14      -0.00264     -0.000529      -0.01510       0.00437  
        2020-01-15       0.00278      0.009670      -0.00675       0.02210  
        2020-01-16       0.01750      0.026100      -0.02120       0.01960  
        2020-01-17      -0.00422      0.010500      -0.03520       0.01580  

[5 rows x 22 columns]

# 2. Subset
## 2.1 California

In [6]:
df.columns

Index(['spend_all', 'spend_aap', 'spend_acf', 'spend_aer', 'spend_apg',
       'spend_durables', 'spend_nondurables', 'spend_grf', 'spend_gen',
       'spend_hic', 'spend_hcs', 'spend_inpersonmisc', 'spend_remoteservices',
       'spend_sgh', 'spend_tws', 'spend_retail_w_grocery',
       'spend_retail_no_grocery', 'spend_all_incmiddle', 'spend_all_q1',
       'spend_all_q2', 'spend_all_q3', 'spend_all_q4'],
      dtype='object')

In [7]:
df_ca_spend = df.loc[idx['California',:], :]
df_ca_spend.index = df_ca_spend.index.droplevel(0)
df_ca_spend = df_ca_spend.resample('m').mean()
#cols = ['spend_all_q1', 'spend_all_q2', 'spend_all_q3', 'spend_all_q4']
df_ca_spend.head(2)

,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,spend_durables,spend_nondurables,spend_grf,spend_gen,spend_hic,...,spend_remoteservices,spend_sgh,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4
date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,0.003406,-0.004683,-0.004202,-0.016351,-0.005010,-0.002897,-0.003472,-0.004228,-0.005799,-0.023406,...,0.007173,-0.007047,0.011334,-0.002158,-0.001432,0.001541,0.004555,0.004286,0.000152,0.004385
2020-02-29,0.015966,-0.016000,-0.001278,0.022367,-0.002055,0.020646,0.012596,0.026260,0.015137,0.101583,...,0.018618,-0.015974,-0.015099,0.016332,0.012900,0.019230,0.018412,0.026724,0.015450,0.014063


## 2.2 NYC

In [8]:
df_ny_spend = df.loc[idx['New York',:], :]
df_ny_spend.index = df_ny_spend.index.droplevel(0)
df_ny_spend = df_ny_spend.resample('m').mean()
cols = ['spend_all_q1', 'spend_all_q2', 'spend_all_q3', 'spend_all_q4']
df_ny_spend.head(2)

,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,spend_durables,spend_nondurables,spend_grf,spend_gen,spend_hic,...,spend_remoteservices,spend_sgh,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4
date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,0.010113,-0.001792,0.005864,0.015677,0.001633,0.008856,0.007135,0.010707,-0.001198,0.044734,...,0.014298,0.011510,0.030337,0.008075,0.006591,0.008461,0.004676,0.008868,0.008148,0.012490
2020-02-29,-0.017713,-0.015555,-0.038213,0.011654,-0.018449,-0.006475,-0.014062,-0.007540,-0.020980,-0.020487,...,0.002756,0.014593,-0.043865,-0.010937,-0.012498,-0.018549,-0.033236,-0.025229,-0.013074,-0.013333
